In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import linregress
from sklearn.preprocessing import MinMaxScaler

# =======================================================
# 1. 파일 불러오기
# =======================================================
print("📂 데이터 로드 중...")

# (1) 원본 VIP 데이터 파일명 (상림이가 가지고 있는 파일명으로 수정해줘!)
# 예: vip_customers.csv
df = pd.read_csv("vip_customers.csv")

# (2) 150개 선정 변수 리스트 파일 (방금 업로드한 파일!)
#
features_df = pd.read_csv("vip_Final_Selected_150_Features_SHAP_Priority.csv")
selected_features = features_df['변수명'].tolist()

print(f"✅ 데이터 크기: {df.shape}")
print(f"✅ 선정된 변수: {len(selected_features)}개")

# =======================================================
# 2. 6개월 추세(Slope) 계산 함수
# =======================================================
def calc_slope_6m(series):
    # 데이터가 부족하거나 전부 0이면 계산 불가
    if len(series) < 2 or np.sum(series) == 0:
        return 0
    
    y = series.values
    x = np.arange(len(y))
    
    # 값이 변하지 않으면 기울기 0
    if np.all(y == y[0]): return 0
    
    # 선형 회귀로 기울기 계산
    slope, _, _, _, _ = linregress(x, y)
    
    # 계산 오류(NaN) 방지
    if np.isnan(slope): return 0
    
    return slope

# =======================================================
# 3. Churn Score (이탈 점수) 만들기
# =======================================================
print("\n⚡ 이탈 점수(Churn_Score) 생성 시작...")

# (1) 데이터 정렬 (고객별, 날짜순 -> 추세 계산을 위해 필수!)
df = df.sort_values(by=['발급회원번호', '기준년월'])

# (2) 주요 지표 3가지 기울기 계산
print("   - 소비/잔액/빈도 추세 계산 중...")
df['Slope_Spend'] = df.groupby('발급회원번호')['이용금액_신용_B0M'].transform(calc_slope_6m)
df['Slope_Count'] = df.groupby('발급회원번호')['이용건수_신용_B0M'].transform(calc_slope_6m)

# 잔액 컬럼명 자동 찾기 (혹시 이름 다를까봐)
balance_col = '잔액_B0M' if '잔액_B0M' in df.columns else [c for c in df.columns if '잔액' in c][0]
df['Slope_Balance'] = df.groupby('발급회원번호')[balance_col].transform(calc_slope_6m)

# (3) 가장 최근 월(Target Month)만 남기기
target_month = df['기준년월'].max()
df_final = df[df['기준년월'] == target_month].copy()

# (4) 기울기를 점수(0~1)로 변환 (감소폭이 클수록 위험!)
scaler = MinMaxScaler()
def convert_slope_to_score(df_in, col_name):
    # 음수(감소)인 경우만 양수로 뒤집어서 점수화
    negative_slopes = df_in[col_name].apply(lambda x: -x if x < 0 else 0)
    return scaler.fit_transform(negative_slopes.values.reshape(-1, 1)).flatten()

score_spend = convert_slope_to_score(df_final, 'Slope_Spend')
score_balance = convert_slope_to_score(df_final, 'Slope_Balance')
score_count = convert_slope_to_score(df_final, 'Slope_Count')

# (5) 가중치 적용 & 최종 점수 합산 (100점 만점)
# 소비(40) + 잔액(30) + 빈도(20) + 리스크(10)
has_risk = np.where(
    (df_final.get('연체잔액_B0M', 0) > 0) | (df_final.get('승인거절건수_B0M', 0) > 0), 
    1, 0
)

df_final['Churn_Score'] = (
    (score_spend * 40) + 
    (score_balance * 30) + 
    (score_count * 20) + 
    (has_risk * 10)
)
df_final['Churn_Score'] = df_final['Churn_Score'].round(1)

print(f"✅ 점수 생성 완료! (평균: {df_final['Churn_Score'].mean():.1f}점)")

# =======================================================
# 4. 150개 변수만 추출 & 저장
# =======================================================
print("\n✂️ 150개 핵심 변수 추출 및 저장...")

# 최종 저장 컬럼: [ID] + [150개 변수] + [Target]
# (데이터에 실제로 존재하는 변수만 교집합으로 선택)
valid_cols = [col for col in selected_features if col in df_final.columns]
final_cols = ['발급회원번호'] + valid_cols + ['Churn_Score']

df_export = df_final[final_cols]

# 파일 저장
save_name = "VIP_Target_150_Selected.csv"
df_export.to_csv(save_name, index=False, encoding='utf-8-sig')

print(f"🎉 파일 저장 완료: {save_name}")